In [1]:
cd ..

/Users/muhammadfarooq/Codebase/wood-log-detection/Pith2Bark


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scikitplot as skplt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Damage Dataset

In [3]:
def read_damage_min_max(damage_fname):
    df_damage = pd.read_csv(damage_fname)
    
    columns = ['min', 'max', 'mean', 'orig', 'diff_min_max', 'diff_orig_mean', 'damage_mean']
    df_damage_temp = df_damage.drop(columns=columns)
    df_damage_temp.rename(columns={'damage_min_max': 'Damage'}, inplace=True)

    df_damage_temp.reset_index(drop=True, inplace=True)

    return df_damage_temp

In [4]:
def read_damage_mean(damage_fname):
    df_damage = pd.read_csv(damage_fname)
    
    columns = ['min', 'max', 'mean', 'orig', 'diff_min_max', 'diff_orig_mean', 'damage_min_max']
    df_damage_temp = df_damage.drop(columns=columns)
    df_damage_temp.rename(columns={'damage_mean': 'Damage'}, inplace=True)

    df_damage_temp.reset_index(drop=True, inplace=True)

    return df_damage_temp

In [5]:
#damage
damage_url = "damage_dataset/damage.csv"

df_damage_min_max = read_damage_min_max(damage_url)

df_damage_mean = read_damage_mean(damage_url)

In [6]:
#STOP

In [7]:
df = df_damage_min_max

In [8]:
# set up the file paths for your image data and labels
img_size = (128, 128)

data_dir = 'damage_dataset/images'

# convert the DataFrame to a dictionary with specific column names as key and value
label_dict = {k: v for k, v in zip(df['image'], df['Damage'])}

# create empty lists to store the image data and labels
images = []
labels = []

# loop through each image file in the directory
for filename in os.listdir(data_dir):
    # load the image file and convert it to a NumPy array
    image = load_img(os.path.join(data_dir, filename), target_size=img_size)
    image = img_to_array(image)
    # preprocess the image by scaling the pixel values to be between 0 and 1
    image = image.astype('float32') / 255.0
    # add the preprocessed image and its label to the lists
    images.append(image)
    labels.append(label_dict[filename])

# convert the lists to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# print the shape of the data to verify that it's been loaded correctly
print('Image data shape:', images.shape) #(148, 64, 64, 3)
print('Label data shape:', labels.shape) #(148,)

Image data shape: (148, 128, 128, 3)
Label data shape: (148,)


In [9]:
#STOP

# Config

In [10]:
training_percent = 0.9

epochs, batch_size = 50, 32
kf = KFold(n_splits=3, shuffle=True, random_state=42)

early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=10,
        restore_best_weights=True,
    )

METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'), 
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

In [11]:
def heat_map_normalized(y_true, y_pred):
    
    class_names = ['Not Damage', 'Damaged']
    
    skplt.metrics.plot_confusion_matrix(y_true, y_pred,
                                        figsize=(4,3),
                                        normalize=True)
    
    # Customize axis tick labels
    plt.xticks([0,1], class_names)
    plt.yticks([0,1], class_names)
    
    plt.yticks(rotation=90)
    
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()

In [12]:
def heat_map(_cm):
    # plot confusion matrix as heatmap
    labels = ['Not Damage', 'Damaged']
    
    # Set up the matplotlib figure
    plt.figure(figsize=(4, 3))
    
    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(230, 20, as_cmap=True)
    
    sns.heatmap(_cm, annot=True, cmap=cmap, xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.show()

In [13]:
units = [128]
activations = ['relu', 'tanh', 'sigmoid']
kernel_size = [3, 5]
learning_rate = [0.001, 0.0001, 0.00001]
optimizer = ['Adam', 'RMSprop', 'Nadam']
dropout = [0.1, 0.2, 0.3]

In [14]:
#STOP

# Model

In [15]:
class Baseline():
    def __init__(self, _input_dim, units, activation, kernel_size, learning_rate, optimizer, dropout, metrics=METRICS):
        # Define the model architecture
        model = Sequential()
        
        # convolutional layer
        model.add(Conv2D(units, kernel_size=kernel_size, activation=activation, input_shape=_input_dim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(dropout))
                  
        # flatten output of conv
        model.add(Flatten())
                  
        # hidden layer
        model.add(Dense(units, activation=activation))
        model.add(Dropout(dropout))
                  
        # output layer
        model.add(Dense(1, activation='sigmoid'))

        if optimizer == 'Adam':
            ops = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        elif optimizer == 'RMSprop':
            ops = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
        elif optimizer == 'Nadam':
            ops = tf.keras.optimizers.Nadam(learning_rate=learning_rate)
            
        # Compile the model
        model.compile(loss='binary_crossentropy', optimizer=ops, metrics=['accuracy'])

        self.model = model

    def train(self, X_train, y_train, X_val, y_val, epochs, batch_size):
        history = self.model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[early_stopping], verbose=2)
        return history
    
    def evaluate(self, X_test, y_test):
        results = self.model.evaluate(X_test, y_test, verbose=0)
        return results

    def predict(self, X_test):
        predictions = self.model.predict(X_test)
        return predictions

    def summary(self):
        self.model.summary()
        
    def metrics_names(self):
        return self.model.metrics_names

# Prepare dataset

In [16]:
X = images
y = labels

# split the data into a holdout set and the rest for K-Fold cross-validation
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, train_size=training_percent, random_state=42)

In [17]:
best_acc = 0.0
baseline_values = { 'units': 0, 'activation': '', 'kernel_size': 0, 'learning_rate' : 0.0, 'optimizer': '', 'dropout': 0.0 }

In [18]:
input_dim = X_train.shape[1:]

In [ ]:
for u in units:
    for a in activations:
        for k in kernel_size:
            for l in learning_rate:
                for o in optimizer:
                    for d in dropout:
                        print('\nUnits:', u, 'Activation:', a, 'Kernel size:', k, 'Learning rate:', l, 'Optimizer:', o, 'Dropout:', d)
                        for train_index, val_index in kf.split(X_train):
                            # split the dataset into training and validation sets for this fold
                            X_train_kf, X_val_kf = X_train[train_index], X_train[val_index]
                            y_train_kf, y_val_kf = y_train[train_index], y_train[val_index]
                            
                            model = Baseline(input_dim, u, a, k, l, o, d)

                            history = model.train(X_train_kf, y_train_kf, X_val_kf, y_val_kf, epochs, batch_size)
                            if (history.history['val_accuracy'][-1] > best_acc):
                                best_acc = history.history['val_accuracy'][-1]
                                baseline_values = { 'units': u, 'activation': a, 'kernel_size': k, 'learning_rate' : l, 'optimizer': o, 'dropout': d }


Units: 64 Activation: relu Kernel size: 3 Learning rate: 0.001 Optimizer: Adam Dropout: 0.1
Epoch 1/50
3/3 - 2s - loss: 3.5960 - accuracy: 0.5568 - val_loss: 4.1422 - val_accuracy: 0.6444 - 2s/epoch - 617ms/step
Epoch 2/50
3/3 - 1s - loss: 3.0855 - accuracy: 0.7045 - val_loss: 2.1419 - val_accuracy: 0.7556 - 898ms/epoch - 299ms/step
Epoch 3/50
3/3 - 1s - loss: 1.6739 - accuracy: 0.7386 - val_loss: 3.4638 - val_accuracy: 0.7778 - 867ms/epoch - 289ms/step
Epoch 4/50
3/3 - 1s - loss: 1.5407 - accuracy: 0.7955 - val_loss: 2.2017 - val_accuracy: 0.7556 - 916ms/epoch - 305ms/step
Epoch 5/50
3/3 - 1s - loss: 1.0014 - accuracy: 0.8523 - val_loss: 0.7731 - val_accuracy: 0.8000 - 932ms/epoch - 311ms/step
Epoch 6/50
3/3 - 1s - loss: 0.7557 - accuracy: 0.8750 - val_loss: 0.4935 - val_accuracy: 0.8667 - 897ms/epoch - 299ms/step
Epoch 7/50
3/3 - 1s - loss: 0.5613 - accuracy: 0.8523 - val_loss: 0.3312 - val_accuracy: 0.8667 - 883ms/epoch - 294ms/step
Epoch 8/50
3/3 - 1s - loss: 0.3387 - accuracy: 0.

Epoch 7/50
3/3 - 1s - loss: 0.3285 - accuracy: 0.8864 - val_loss: 0.3982 - val_accuracy: 0.8667 - 1s/epoch - 352ms/step
Epoch 8/50
3/3 - 1s - loss: 0.4123 - accuracy: 0.8523 - val_loss: 0.4349 - val_accuracy: 0.8000 - 1s/epoch - 431ms/step
Epoch 9/50
3/3 - 1s - loss: 0.3175 - accuracy: 0.8864 - val_loss: 0.4320 - val_accuracy: 0.8000 - 1s/epoch - 452ms/step
Epoch 10/50
3/3 - 1s - loss: 0.2655 - accuracy: 0.8977 - val_loss: 0.3983 - val_accuracy: 0.8000 - 1s/epoch - 364ms/step
Epoch 11/50
3/3 - 1s - loss: 0.2000 - accuracy: 0.9091 - val_loss: 0.3844 - val_accuracy: 0.8667 - 1s/epoch - 351ms/step
Epoch 12/50
3/3 - 1s - loss: 0.1898 - accuracy: 0.9091 - val_loss: 0.3682 - val_accuracy: 0.8667 - 1s/epoch - 343ms/step
Epoch 13/50
3/3 - 1s - loss: 0.1668 - accuracy: 0.9545 - val_loss: 0.3849 - val_accuracy: 0.8222 - 1s/epoch - 354ms/step
Epoch 14/50
3/3 - 1s - loss: 0.1728 - accuracy: 0.9432 - val_loss: 0.4220 - val_accuracy: 0.7778 - 1s/epoch - 335ms/step
Epoch 15/50
3/3 - 1s - loss: 0.1286

Epoch 17/50
3/3 - 1s - loss: 0.1486 - accuracy: 0.9318 - val_loss: 0.3992 - val_accuracy: 0.8667 - 940ms/epoch - 313ms/step
Epoch 18/50
3/3 - 1s - loss: 0.1254 - accuracy: 0.9545 - val_loss: 0.4364 - val_accuracy: 0.8000 - 961ms/epoch - 320ms/step
Epoch 19/50
3/3 - 1s - loss: 0.1135 - accuracy: 1.0000 - val_loss: 0.5743 - val_accuracy: 0.7778 - 926ms/epoch - 309ms/step
Epoch 20/50
3/3 - 1s - loss: 0.1411 - accuracy: 0.9545 - val_loss: 0.5320 - val_accuracy: 0.8000 - 946ms/epoch - 315ms/step
Epoch 1/50
3/3 - 2s - loss: 4.4223 - accuracy: 0.6067 - val_loss: 2.0315 - val_accuracy: 0.8409 - 2s/epoch - 720ms/step
Epoch 2/50
3/3 - 1s - loss: 1.5809 - accuracy: 0.8202 - val_loss: 5.9829 - val_accuracy: 0.6364 - 1s/epoch - 374ms/step
Epoch 3/50
3/3 - 1s - loss: 2.8112 - accuracy: 0.8202 - val_loss: 2.5270 - val_accuracy: 0.8409 - 1s/epoch - 418ms/step
Epoch 4/50
3/3 - 1s - loss: 1.1149 - accuracy: 0.8090 - val_loss: 1.8792 - val_accuracy: 0.8182 - 1s/epoch - 466ms/step
Epoch 5/50
3/3 - 1s - lo

3/3 - 1s - loss: 0.3881 - accuracy: 0.8636 - val_loss: 0.3644 - val_accuracy: 0.8667 - 1s/epoch - 348ms/step
Epoch 15/50
3/3 - 1s - loss: 0.3769 - accuracy: 0.8409 - val_loss: 0.3889 - val_accuracy: 0.8667 - 944ms/epoch - 315ms/step
Epoch 16/50
3/3 - 1s - loss: 0.4404 - accuracy: 0.8523 - val_loss: 0.4498 - val_accuracy: 0.7556 - 961ms/epoch - 320ms/step
Epoch 17/50
3/3 - 1s - loss: 0.4161 - accuracy: 0.8636 - val_loss: 0.3887 - val_accuracy: 0.8667 - 927ms/epoch - 309ms/step
Epoch 18/50
3/3 - 1s - loss: 0.4207 - accuracy: 0.8523 - val_loss: 0.4871 - val_accuracy: 0.7778 - 932ms/epoch - 311ms/step
Epoch 19/50
3/3 - 1s - loss: 0.4841 - accuracy: 0.8182 - val_loss: 0.3868 - val_accuracy: 0.8444 - 929ms/epoch - 310ms/step
Epoch 20/50
3/3 - 1s - loss: 0.3553 - accuracy: 0.8864 - val_loss: 0.4551 - val_accuracy: 0.7556 - 946ms/epoch - 315ms/step
Epoch 21/50
3/3 - 1s - loss: 0.3909 - accuracy: 0.8636 - val_loss: 0.4112 - val_accuracy: 0.8222 - 948ms/epoch - 316ms/step
Epoch 22/50
3/3 - 1s - 

Epoch 13/50
3/3 - 1s - loss: 0.3414 - accuracy: 0.8764 - val_loss: 0.8040 - val_accuracy: 0.5000 - 1s/epoch - 341ms/step
Epoch 14/50
3/3 - 1s - loss: 0.8273 - accuracy: 0.6404 - val_loss: 2.5545 - val_accuracy: 0.5909 - 1s/epoch - 339ms/step
Epoch 15/50
3/3 - 1s - loss: 0.6121 - accuracy: 0.8652 - val_loss: 0.6908 - val_accuracy: 0.8182 - 1s/epoch - 348ms/step
Epoch 16/50
3/3 - 1s - loss: 0.3757 - accuracy: 0.8876 - val_loss: 0.6501 - val_accuracy: 0.8182 - 1s/epoch - 345ms/step
Epoch 1/50
3/3 - 2s - loss: 11.6907 - accuracy: 0.5506 - val_loss: 0.8970 - val_accuracy: 0.8636 - 2s/epoch - 609ms/step
Epoch 2/50
3/3 - 1s - loss: 1.9780 - accuracy: 0.7079 - val_loss: 0.9765 - val_accuracy: 0.8636 - 1s/epoch - 356ms/step
Epoch 3/50
3/3 - 1s - loss: 2.0698 - accuracy: 0.7303 - val_loss: 0.3438 - val_accuracy: 0.9091 - 1s/epoch - 385ms/step
Epoch 4/50
3/3 - 1s - loss: 1.8093 - accuracy: 0.6404 - val_loss: 0.5175 - val_accuracy: 0.8636 - 1s/epoch - 347ms/step
Epoch 5/50
3/3 - 1s - loss: 1.2045 

...

Epoch 41/50
3/3 - 1s - loss: 1.2920 - accuracy: 0.4270 - val_loss: 1.6259 - val_accuracy: 0.2500 - 1s/epoch - 386ms/step
Epoch 42/50
3/3 - 1s - loss: 1.2795 - accuracy: 0.4270 - val_loss: 1.6213 - val_accuracy: 0.2500 - 1s/epoch - 386ms/step
Epoch 43/50
3/3 - 1s - loss: 1.3326 - accuracy: 0.4270 - val_loss: 1.6167 - val_accuracy: 0.2500 - 1s/epoch - 379ms/step
Epoch 44/50
3/3 - 1s - loss: 1.3114 - accuracy: 0.4270 - val_loss: 1.6120 - val_accuracy: 0.2500 - 1s/epoch - 387ms/step
Epoch 45/50
3/3 - 1s - loss: 1.2554 - accuracy: 0.4270 - val_loss: 1.6074 - val_accuracy: 0.2500 - 1s/epoch - 381ms/step
Epoch 46/50
3/3 - 1s - loss: 1.3262 - accuracy: 0.4270 - val_loss: 1.6027 - val_accuracy: 0.2500 - 1s/epoch - 380ms/step
Epoch 47/50
3/3 - 1s - loss: 1.3103 - accuracy: 0.4270 - val_loss: 1.5980 - val_accuracy: 0.2500 - 1s/epoch - 380ms/step
Epoch 48/50
3/3 - 1s - loss: 1.2937 - accuracy: 0.4270 - val_loss: 1.5933 - val_accuracy: 0.2500 - 1s/epoch - 385ms/step
Epoch 49/50
3/3 - 1s - loss: 1.2

Epoch 7/50
3/3 - 1s - loss: 0.6377 - accuracy: 0.6966 - val_loss: 0.7466 - val_accuracy: 0.5000 - 950ms/epoch - 317ms/step
Epoch 8/50
3/3 - 1s - loss: 0.6451 - accuracy: 0.6742 - val_loss: 0.7316 - val_accuracy: 0.5000 - 1s/epoch - 383ms/step
Epoch 9/50
3/3 - 1s - loss: 0.6471 - accuracy: 0.6742 - val_loss: 0.7341 - val_accuracy: 0.5000 - 966ms/epoch - 322ms/step
Epoch 10/50
3/3 - 1s - loss: 0.6099 - accuracy: 0.7191 - val_loss: 0.7443 - val_accuracy: 0.5000 - 1s/epoch - 347ms/step
Epoch 11/50
3/3 - 1s - loss: 0.6562 - accuracy: 0.6742 - val_loss: 0.7558 - val_accuracy: 0.5000 - 942ms/epoch - 314ms/step
Epoch 12/50
3/3 - 1s - loss: 0.6070 - accuracy: 0.6966 - val_loss: 0.7624 - val_accuracy: 0.5000 - 943ms/epoch - 314ms/step
Epoch 1/50
3/3 - 2s - loss: 0.9407 - accuracy: 0.5056 - val_loss: 0.5738 - val_accuracy: 0.7500 - 2s/epoch - 630ms/step
Epoch 2/50
3/3 - 1s - loss: 0.7017 - accuracy: 0.5618 - val_loss: 0.8810 - val_accuracy: 0.2500 - 986ms/epoch - 329ms/step
Epoch 3/50
3/3 - 1s - 

Epoch 48/50
3/3 - 1s - loss: 0.5469 - accuracy: 0.7386 - val_loss: 0.5899 - val_accuracy: 0.7333 - 1s/epoch - 353ms/step
Epoch 49/50
3/3 - 1s - loss: 0.6204 - accuracy: 0.7045 - val_loss: 0.5892 - val_accuracy: 0.7778 - 990ms/epoch - 330ms/step
Epoch 50/50
3/3 - 1s - loss: 0.5550 - accuracy: 0.7841 - val_loss: 0.5834 - val_accuracy: 0.7556 - 975ms/epoch - 325ms/step
Epoch 1/50
3/3 - 2s - loss: 0.9013 - accuracy: 0.6292 - val_loss: 0.9397 - val_accuracy: 0.5000 - 2s/epoch - 645ms/step
Epoch 2/50
3/3 - 1s - loss: 0.6160 - accuracy: 0.6517 - val_loss: 0.6922 - val_accuracy: 0.5000 - 1s/epoch - 439ms/step
Epoch 3/50
3/3 - 1s - loss: 0.7205 - accuracy: 0.4157 - val_loss: 0.7835 - val_accuracy: 0.5000 - 1s/epoch - 335ms/step
Epoch 4/50
3/3 - 1s - loss: 0.6724 - accuracy: 0.6966 - val_loss: 0.9844 - val_accuracy: 0.5000 - 977ms/epoch - 326ms/step
Epoch 5/50
3/3 - 1s - loss: 0.6796 - accuracy: 0.6966 - val_loss: 0.8115 - val_accuracy: 0.5000 - 1s/epoch - 393ms/step
Epoch 6/50
3/3 - 1s - loss: 

Epoch 38/50
3/3 - 1s - loss: 0.6312 - accuracy: 0.6705 - val_loss: 0.6235 - val_accuracy: 0.6444 - 1s/epoch - 352ms/step
Epoch 39/50
3/3 - 1s - loss: 0.6360 - accuracy: 0.6477 - val_loss: 0.6281 - val_accuracy: 0.6444 - 970ms/epoch - 323ms/step
Epoch 40/50
3/3 - 1s - loss: 0.6275 - accuracy: 0.6705 - val_loss: 0.6221 - val_accuracy: 0.6444 - 1s/epoch - 348ms/step
Epoch 41/50
3/3 - 1s - loss: 0.6173 - accuracy: 0.6932 - val_loss: 0.6212 - val_accuracy: 0.6444 - 1s/epoch - 356ms/step
Epoch 42/50
3/3 - 1s - loss: 0.6578 - accuracy: 0.6591 - val_loss: 0.6200 - val_accuracy: 0.6444 - 1s/epoch - 345ms/step
Epoch 43/50
3/3 - 1s - loss: 0.6758 - accuracy: 0.5909 - val_loss: 0.6222 - val_accuracy: 0.6444 - 1s/epoch - 338ms/step
Epoch 44/50
3/3 - 1s - loss: 0.6287 - accuracy: 0.6932 - val_loss: 0.6234 - val_accuracy: 0.6444 - 1s/epoch - 343ms/step
Epoch 45/50
3/3 - 1s - loss: 0.6644 - accuracy: 0.5909 - val_loss: 0.6169 - val_accuracy: 0.6444 - 1s/epoch - 366ms/step
Epoch 46/50
3/3 - 1s - loss: 

3/3 - 1s - loss: 0.6760 - accuracy: 0.6364 - val_loss: 0.6522 - val_accuracy: 0.6444 - 1s/epoch - 372ms/step
Epoch 34/50
3/3 - 1s - loss: 0.6756 - accuracy: 0.5795 - val_loss: 0.6466 - val_accuracy: 0.6444 - 1s/epoch - 392ms/step
Epoch 35/50
3/3 - 1s - loss: 0.6337 - accuracy: 0.6250 - val_loss: 0.6464 - val_accuracy: 0.6444 - 1s/epoch - 395ms/step
Epoch 36/50
3/3 - 1s - loss: 0.6807 - accuracy: 0.6364 - val_loss: 0.6488 - val_accuracy: 0.6444 - 1s/epoch - 359ms/step
Epoch 37/50
3/3 - 1s - loss: 0.6564 - accuracy: 0.6136 - val_loss: 0.6468 - val_accuracy: 0.6444 - 1s/epoch - 345ms/step
Epoch 38/50
3/3 - 1s - loss: 0.6433 - accuracy: 0.6023 - val_loss: 0.6476 - val_accuracy: 0.6444 - 1s/epoch - 373ms/step
Epoch 39/50
3/3 - 1s - loss: 0.6504 - accuracy: 0.6477 - val_loss: 0.6391 - val_accuracy: 0.6444 - 1s/epoch - 372ms/step
Epoch 40/50
3/3 - 1s - loss: 0.6181 - accuracy: 0.6250 - val_loss: 0.6469 - val_accuracy: 0.6444 - 1s/epoch - 416ms/step
Epoch 1/50
3/3 - 2s - loss: 0.9343 - accurac

In [ ]:
print(best_acc)
print(baseline_values)

In [ ]:
#STOP